In [1]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document as langchain_Document
from transformers import AutoModel
from numpy.linalg import norm

In [2]:
cache_directory="/data3/.cache"
cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-zh', trust_remote_code=True)

C:\ProgramData\anaconda3\envs\py310\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
file_path = "find_info.xls"

df = pd.read_excel(file_path, usecols="M")

In [4]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [5]:
df

,Feature
0,身上有毛周角化皮肤症状
1,身高一米六三，体重70公斤，国字脸，腰左胯有豆粒大的胎记，单眼皮
2,抱养回家有几个月。
3,寻母亲
4,1.有一条红绳绑着2.4元和生日时辰。2.有一顶粉红帽。3.有一条裤子包着婴儿和西装短裤 。
...,...
42973,男孩，5、6岁样子，1米高左右，详见相片。
42974,没有其他异样，孩子应该是健康的，
42975,短发，神智清楚
42976,一个盲人带着一个女孩在夜市乞讨，说是四川的


In [6]:
# 将 DataFrame 转换为嵌套列表
lst_data = df.fillna("").values.tolist()

In [7]:
metadata_list = [{"idx": idx, "raw_data": data, "content": "\n".join(data).strip()} for idx, data in enumerate(lst_data, 0)]

In [8]:
metadata_list = [{"idx": idx, "raw_data": data, "content": "\n".join(data).strip()} for idx, data in enumerate(lst_data, 0)] 

In [9]:
metadata_list

[{'idx': 0, 'raw_data': ['身上有毛周角化皮肤症状'], 'content': '身上有毛周角化皮肤症状'},
 {'idx': 1,
  'raw_data': ['身高一米六三，体重70公斤，国字脸，腰左胯有豆粒大的胎记，单眼皮'],
  'content': '身高一米六三，体重70公斤，国字脸，腰左胯有豆粒大的胎记，单眼皮'},
 {'idx': 2, 'raw_data': ['抱养回家有几个月。'], 'content': '抱养回家有几个月。'},
 {'idx': 3, 'raw_data': ['寻母亲'], 'content': '寻母亲'},
 {'idx': 4,
  'raw_data': ['1.有一条红绳绑着2.4元和生日时辰。2.有一顶粉红帽。3.有一条裤子包着婴儿和西装短裤 。'],
  'content': '1.有一条红绳绑着2.4元和生日时辰。2.有一顶粉红帽。3.有一条裤子包着婴儿和西装短裤 。'},
 {'idx': 5, 'raw_data': ['左耳前有胎记'], 'content': '左耳前有胎记'},
 {'idx': 6,
  'raw_data': ['脸上有一个洞 就是少了皮肤的那种洞 很小 脖子那有一个痣'],
  'content': '脸上有一个洞 就是少了皮肤的那种洞 很小 脖子那有一个痣'},
 {'idx': 7, 'raw_data': ['走失时15岁小女孩'], 'content': '走失时15岁小女孩'},
 {'idx': 8, 'raw_data': ['圆脸 胖胖的 很白 刚刚出生'], 'content': '圆脸 胖胖的 很白 刚刚出生'},
 {'idx': 9,
  'raw_data': ['49年前母亲抱着我刚出生不久，我身穿黄色衣服（或是马甲）缝纫机走线，外包裹小被孑在甘肃省张掖市刚进城路边的公厕给了一个小女孩抱着，她这一走就再没回来'],
  'content': '49年前母亲抱着我刚出生不久，我身穿黄色衣服（或是马甲）缝纫机走线，外包裹小被孑在甘肃省张掖市刚进城路边的公厕给了一个小女孩抱着，她这一走就再没回来'},
 {'idx': 10, 'raw_data': ['左左脸上有一块小胎记'], 'content': '左左脸上有一块

In [10]:
import numpy as np
from tqdm import tqdm

batch_size = 100  # 根据具体硬件配置来调整这个大小
batches = [metadata_list[i:i + batch_size] for i in range(0, len(metadata_list), batch_size)]

for batch in tqdm(batches, desc="Processing batches"):
    contents = [metadata['content'] for metadata in batch]
    embeddings = model.encode(contents)
    for idx, metadata in enumerate(batch):
        metadata["embedding"] = embeddings[idx]

Processing batches: 100%|██████████| 430/430 [1:19:46<00:00, 11.13s/it]


In [11]:
type(metadata_list[0]["embedding"])

numpy.ndarray

In [12]:
import pickle

# 使用 pickle 写入文件
with open("data.pkl", "wb") as file:
    pickle.dump(metadata_list, file)

In [13]:
import sys
def convert_size(size_bytes):
    """将字节为单位的大小转换为更合适的单位（KB, MB, GB等）。"""
    if size_bytes == 0:
        return "0B"
    size_names = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(np.floor(np.log(size_bytes) / np.log(1024)))
    p = np.power(1024, i)
    s = round(size_bytes / p, 2)
    return f"{s} {size_names[i]}"
convert_size(sys.getsizeof(metadata_list))

'342.84 KB'

In [14]:
import numpy as np

input_embedding = model.encode("黑痣")
embeddings = np.array([item['embedding'] for item in metadata_list])


In [15]:
from scipy.spatial.distance import cdist

distances = cdist([input_embedding], embeddings, 'cosine')[0]

In [16]:
# 获取最匹配的 k 个条目的索引
k = 10  # 匹配数量
closest_indices = np.argsort(distances)[:k]

# 返回这些条目的 idx 和相似度
closest_idxs_and_similarities = [(metadata_list[idx]['idx'], 1 - distances[idx]) for idx in closest_indices]
for idx, similarity in closest_idxs_and_similarities:
    print(f"Idx: {idx}, Similarity: {similarity:.4f}")
    print(metadata_list[idx]["content"])

Idx: 34017, Similarity: 0.8407
脸有颗黑痣
Idx: 38375, Similarity: 0.8150
脸旁有颗黑痣
Idx: 42115, Similarity: 0.8107
左脸有一黑痣
Idx: 42680, Similarity: 0.8079
屁股上有颗黑痣
Idx: 405, Similarity: 0.8066
后背有一黑痣
Idx: 30856, Similarity: 0.8062
背部有一个黑痣
Idx: 42355, Similarity: 0.7995
后背有个黑痣
Idx: 4016, Similarity: 0.7993
左背有黑痣
Idx: 34318, Similarity: 0.7987
后背有颗黑痣
Idx: 17373, Similarity: 0.7986
人中上有一只黑痣


In [17]:
documents=[langchain_Document(page_content=metadata["content"], metadata=metadata) for
               metadata in metadata_list]

In [18]:
documents=documents[:100]

In [29]:
db = FAISS.from_documents(
    documents=documents,
    embedding=utils_daoer.JinaEmbeddings(url="http://127.0.0.1:50086/api/v2/embed_onnx")
)

NameError: name 'utils_daoer' is not defined

In [ ]:
db.save_local("./faiss_index")
